# Weighting Minimax Risk (WMMR)

In [1]:
import os
import sys
sys.path.append('..')
import yaml
import shutil
import argparse
import numpy as np
from tqdm import tqdm
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, random_split

from utils import *

## Parameter setting

In [2]:
gpu = '4,5,6,7'
dataset = 'cifar10'
model_type = 'wrn34-10'
checkpoint = './checkpoint/wmmr/%s/%s' % (model_type, dataset)
num_classes = 10
lr = 0.2
weight_decay = 5e-4
batch_size = 128
total_epochs = 100
tau = 0.5
epsilon = 8/255
alpha = 2/255
num_repeats = 10

## Inner maximization

In [3]:
def inner_max(model, inputs, targets, epsilon, alpha, num_repeats, tau):
    noise = torch.FloatTensor(inputs.shape).normal_(0, 1).cuda()
    x = torch.clamp(inputs + 0.001*noise, min=0, max=1)
    batch = x.size(0)
    model.eval()
    for _ in range(num_repeats):
        x.requires_grad_()
        with torch.enable_grad():
            logits = model(x)
        
        classes = torch.arange(logits.size(1))[None,:].repeat(batch,1).cuda()
        false_probs = logits.softmax(dim=1)[classes!=targets[:,None]].view(batch, logits.size(1)-1)
        gt_probs = logits.softmax(dim=1)[classes==targets[:,None]].unsqueeze(1)
        top2_probs = torch.topk(false_probs, k=1).values
        s = torch.exp(-tau*(gt_probs - top2_probs)).view(-1)
        
        loss = -torch.sum(s*torch.log_softmax(logits, dim=1)[classes==targets[:,None]])/batch
        loss.backward()
        grads = x.grad.data
        x = x.data.detach() + alpha*torch.sign(grads).detach()
        x = torch.min(torch.max(x, inputs-epsilon), inputs+epsilon).clamp(min=0, max=1)
    return x

## Training (Outer minimization)

In [4]:
def training(epoch, model, dataloader, optimizer, num_classes, 
             tau=2.0, epsilon=8/255, alpha=2/255, num_repeats=10):
    model.train()
    total = 0
    total_loss = 0
    total_correct = 0
        
    for idx, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        batch = inputs.size(0)
        
        x = inner_max(model, inputs, targets, epsilon, alpha, num_repeats, tau)
        model.train()
        logits = model(x)
        
        classes = torch.arange(num_classes)[None,:].repeat(batch,1).cuda()
        false_probs = logits.softmax(dim=1)[classes!=targets[:,None]].view(batch, logits.size(1)-1)
        gt_probs = logits.softmax(dim=1)[classes==targets[:,None]].unsqueeze(1)
        top2_probs = torch.topk(false_probs, k=1).values
        s = torch.exp(-tau*(gt_probs - top2_probs)).view(-1)
        
        loss = -torch.sum(s*torch.log_softmax(logits, dim=1)[classes==targets[:,None]])/batch
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
        total += batch
        total_loss += loss.item()
        num_correct = torch.argmax(logits.data, dim=1).eq(targets.data).cpu().sum().item()
        total_correct += num_correct
        
        if idx % 100 == 0:
            print('Epoch %d [%d/%d] | loss: %.4f (avg: %.4f) | acc: %.4f (avg: %.4f) |'\
                  % (epoch, idx, len(dataloader), loss.item(), total_loss/(idx+1),
                     num_correct/batch, total_correct/total))

In [5]:
def evaluation(epoch, model, dataloader, alpha, epsilon, num_repeats):
    model.eval()
    total_correct_nat = 0
    total_correct_adv = 0
    
    xent = nn.CrossEntropyLoss()
    with tqdm(dataloader) as pbar:
        for index, (inputs, targets) in enumerate(pbar):
            inputs, targets = inputs.cuda(), targets.cuda()
            batch = inputs.size(0)
            pbar.set_description("[eval] %d/%d" % (index+1, len(dataloader.dataset)))
            
            noise = torch.FloatTensor(inputs.shape).uniform_(-1, 1).cuda()
            x = torch.clamp(inputs+noise, min=0, max=1)
            for _ in range(num_repeats):
                x.requires_grad_()
                with torch.enable_grad():
                    logits = model(x)
                loss = xent(logits, targets)
                loss.backward()
                grads = x.grad.data
                x = x.data.detach() + alpha*torch.sign(grads).detach()
                x = torch.min(torch.max(x, inputs-epsilon), inputs+epsilon).clamp(min=0,max=1)
                
            delta = x - inputs
            x = torch.clamp(inputs+delta, min=0, max=1)
            with torch.no_grad():
                logits_nat = model(inputs)
                logits_adv = model(x)
            total_correct_nat += torch.argmax(logits_nat.data, dim=1).eq(targets.data).cpu().sum().item()
            total_correct_adv += torch.argmax(logits_adv.data, dim=1).eq(targets.data).cpu().sum().item()
            pbar.set_postfix(OrderedDict(acc_nat=total_correct_nat / len(dataloader.dataset),
                                         acc_rob=total_correct_adv / len(dataloader.dataset)))
    
    return (total_correct_nat / len(dataloader.dataset)), (total_correct_adv / len(dataloader.dataset))

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = gpu
os.makedirs(checkpoint, exist_ok=True)

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()])
train_dataset, _ = get_dataloader(dataset, batch_size)
num_samples = len(train_dataset)
num_samples_for_train = int(num_samples * 0.98)
num_samples_for_valid = num_samples - num_samples_for_train
train_set, valid_set = random_split(train_dataset, [num_samples_for_train, num_samples_for_valid])
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=False)
valid_dataloader = DataLoader(valid_set, batch_size=1, shuffle=True, drop_last=False)

model = nn.DataParallel(get_network(model_type, num_classes).cuda())
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)

scheduler = [75, 90]
adjust_learning_rate = lr_scheduler.MultiStepLR(optimizer, scheduler, gamma=0.1)
best_acc_nat, best_acc_rob = 0, 0

for epoch in range(total_epochs):
    training(epoch, model, train_dataloader, optimizer, num_classes, tau, epsilon, alpha, num_repeats)
    test_acc_nat, test_acc_rob = evaluation(epoch, model, valid_dataloader, alpha, epsilon, num_repeats)

    is_best = best_acc_nat < test_acc_nat and best_acc_rob < test_acc_rob
    best_acc_nat = max(best_acc_nat, test_acc_nat)
    best_acc_rob = max(best_acc_rob, test_acc_rob)
    save_checkpoint = {'state_dict': model.state_dict(),
                       'best_acc_nat': best_acc_nat,
                       'best_acc_rob': best_acc_rob,
                       'optimizer': optimizer.state_dict(),
                       'model_type': model_type,
                       'dataset': dataset}
    torch.save(save_checkpoint, os.path.join(checkpoint, 'model'))
    if is_best:
        torch.save(save_checkpoint, os.path.join(checkpoint, 'best_model'))
    adjust_learning_rate.step()

Files already downloaded and verified
Files already downloaded and verified
Epoch 0 [0/383] | loss: 2.3725 (avg: 2.3725) | acc: 0.1016 (avg: 0.1016) |
Epoch 0 [100/383] | loss: 2.3097 (avg: 3.3160) | acc: 0.1016 (avg: 0.1069) |
Epoch 0 [200/383] | loss: 2.2830 (avg: 2.8140) | acc: 0.1328 (avg: 0.1103) |
Epoch 0 [300/383] | loss: 2.2870 (avg: 2.6386) | acc: 0.1562 (avg: 0.1199) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:17<00:00,  5.05it/s, acc_nat=0.161, acc_rob=0.133]


Epoch 1 [0/383] | loss: 2.2590 (avg: 2.2590) | acc: 0.1484 (avg: 0.1484) |
Epoch 1 [100/383] | loss: 2.2301 (avg: 2.2473) | acc: 0.2109 (avg: 0.1726) |
Epoch 1 [200/383] | loss: 2.1746 (avg: 2.2452) | acc: 0.1562 (avg: 0.1729) |
Epoch 1 [300/383] | loss: 2.1588 (avg: 2.2427) | acc: 0.2109 (avg: 0.1726) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:19<00:00,  5.02it/s, acc_nat=0.21, acc_rob=0.172]


Epoch 2 [0/383] | loss: 2.1794 (avg: 2.1794) | acc: 0.1719 (avg: 0.1719) |
Epoch 2 [100/383] | loss: 2.2136 (avg: 2.2152) | acc: 0.1641 (avg: 0.1876) |
Epoch 2 [200/383] | loss: 2.1638 (avg: 2.2053) | acc: 0.2031 (avg: 0.1913) |
Epoch 2 [300/383] | loss: 2.1612 (avg: 2.2028) | acc: 0.2031 (avg: 0.1942) |


...

Epoch 34 [0/383] | loss: 1.7648 (avg: 1.7648) | acc: 0.3828 (avg: 0.3828) |
Epoch 34 [100/383] | loss: 1.5642 (avg: 1.5936) | acc: 0.4531 (avg: 0.4345) |
Epoch 34 [200/383] | loss: 1.4669 (avg: 1.5946) | acc: 0.5469 (avg: 0.4390) |
Epoch 34 [300/383] | loss: 1.8151 (avg: 1.6002) | acc: 0.2969 (avg: 0.4378) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:19<00:00,  5.01it/s, acc_nat=0.63, acc_rob=0.416]


Epoch 35 [0/383] | loss: 1.6968 (avg: 1.6968) | acc: 0.3906 (avg: 0.3906) |
Epoch 35 [100/383] | loss: 1.4377 (avg: 1.5874) | acc: 0.5078 (avg: 0.4411) |
Epoch 35 [200/383] | loss: 1.5733 (avg: 1.5880) | acc: 0.4922 (avg: 0.4415) |
Epoch 35 [300/383] | loss: 1.4914 (avg: 1.5942) | acc: 0.5078 (avg: 0.4416) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:18<00:00,  5.05it/s, acc_nat=0.607, acc_rob=0.345]


Epoch 36 [0/383] | loss: 1.6058 (avg: 1.6058) | acc: 0.5000 (avg: 0.5000) |
Epoch 36 [100/383] | loss: 1.6855 (avg: 1.6058) | acc: 0.4062 (avg: 0.4397) |
Epoch 36 [200/383] | loss: 1.4447 (avg: 1.5928) | acc: 0.4844 (avg: 0.4485) |
Epoch 36 [300/383] | loss: 1.7280 (avg: 1.6073) | acc: 0.3750 (avg: 0.4419) |
